# 準備

## ライブラリ

In [33]:
!pip install --upgrade xlrd==1.2.0
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import urllib.parse
import urllib.request
import json
import os
import pandas as pd
import numpy as np
import xlrd
import openpyxl

## 都道府県一覧

[RESAS-API](https://opendata.resas-portal.go.jp/docs/api/v1/prefectures.html)を利用する

利用者登録した後、APIキーを設定

In [34]:
RESAS_API_KEY = '02JYzLqUOfTNVfiTWjLlR2g0YwEQPQ7caYxcnZaZ'

In [35]:
def preflist():
  url = 'https://opendata.resas-portal.go.jp/api/v1/prefectures'
  req = urllib.request.Request(url, headers={'X-API-KEY': RESAS_API_KEY})
  with urllib.request.urlopen(req) as response:
    res = json.loads(response.read().decode())
  
  return  pd.DataFrame(res['result'])

In [36]:
preflist().head()

,prefCode,prefName
0,1,北海道
1,2,青森県
2,3,岩手県
3,4,宮城県
4,5,秋田県


## 市区町村一覧

同じく[RESAS-API](https://opendata.resas-portal.go.jp/docs/api/v1/cities.html)を利用する

In [30]:
def citylist(prefCode=None):
    url = 'https://opendata.resas-portal.go.jp/api/v1/cities'
    req = urllib.request.Request(url, headers={'X-API-KEY': RESAS_API_KEY})
    with urllib.request.urlopen(req) as response:
        res = json.loads(response.read().decode())
    
    res = list(filter(lambda x: x['bigCityFlag'] != '1', res['result']))

    if prefCode:
      res = list(filter(lambda x: x['prefCode'] == prefCode, res))
    
    return pd.DataFrame(res)

In [31]:
# 全国の市区町村
citylist().head()

,prefCode,cityCode,cityName,bigCityFlag
0,1,01100,札幌市,2
1,1,01202,函館市,0
2,1,01203,小樽市,0
3,1,01204,旭川市,0
4,1,01205,室蘭市,0


In [32]:
# 兵庫県の市区町村
citylist(28).head()

,prefCode,cityCode,cityName,bigCityFlag
0,28,28100,神戸市,2
1,28,28201,姫路市,0
2,28,28202,尼崎市,0
3,28,28203,明石市,0
4,28,28204,西宮市,0


## estat-APIのデータ取得

[政府統計の総合窓口](https://www.e-stat.go.jp/)のAPIからデータを取得する関数

利用者登録した後、APP_IDを設定

In [38]:
ESTAT_APP_ID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

In [44]:
import urllib.parse
import urllib.request
import json

def get_estat_response(params):
  # appId
  params['appId'] = ESTAT_APP_ID
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(params)

  # API取得
  with urllib.request.urlopen(url) as response:
    res = json.loads(response.read().decode('utf-8'))
  
  # DataFrameに変換
  VALUE = res['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']
  df = pd.DataFrame(VALUE)

  return df

例えば総人口データの取得は次のとおり

In [45]:
params = {
    'statsDataId': '0000010101',
    'cdCat01': 'A1101',
    }
get_estat_response(params).head()

,@tab,@cat01,@area,@time,@unit,$
0,00001,A1101,00000,1975100000,人,111939643
1,00001,A1101,00000,1976100000,人,113094000
2,00001,A1101,00000,1977100000,人,114165000
3,00001,A1101,00000,1978100000,人,115190000
4,00001,A1101,00000,1979100000,人,116155000


# 1 計画策定の目的

# 2 播磨臨海地域の特徴

## 2-1 全国屈指のものづくり拠点

市区町村の製造品出荷額

[工業統計調査](https://www.e-stat.go.jp/dbview?sid=0003448126)の2020年確報データを利用する

In [48]:
def manufacture():
  params = {
      'statsDataId' : '0003448126',
      'cdCat01' : '32000120',
      'cdCat02' : '00',
      }
  
  df = get_estat_response(params).head()
  
  df_city = citylist()
  df = pd.merge()

  return df


In [50]:
manufacture().head()

,@cat01,@cat02,@cat03,$
0,32000120,00,01,604889350
1,32000120,00,01100,58960608
2,32000120,00,01101,2136381
3,32000120,00,01102,2040838
4,32000120,00,01103,11747635


## 2-2 姫路港・東播磨港の取扱貨物

# 3 基本的な事項

# 4 港湾地域の面的・効率的な脱炭素化


# 5 水素・燃料アンモニア等供給目標及び供給計画